# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

TF-IDF计算公式如下：
$$TF-IDF(x)=TF(x) \times IDF(x)$$  

其中$TF(x)$为词*x*在文档*d*中出现的频率，即词频，计算公式为：  
$$TF(x)=\frac{count(x \space occurrence)}{count(Total \space words \space in \space Document)}$$  

由于对于文档*d*来说，分母对于每个词都是一样的，所以可以省略分母，并增加log变换使各词的计算结果差值变小，变成以下形式：
$$TF(x)=log(count(x \space occurrence))$$  

*TF还有一种常见的形式，分母使用该文档中出现最多的词的出现次数，对应形式为:$TF(x)=\frac{count(x \space occurrence)}{count(most \space frequent \space word \space occurrence)}$*


$IDF(x)$为词*x*的逆文档频率（Inverse Document Frequency），通过用文档总数除以包含词*x*的文档数目得到，使用加一平滑避免除0的问题，并在结果上增加log变换，其公式为：
$$IDF(x)=log(\frac{count(Documents) + 1}{count(Documents \space with \space x \space occurred) + 1}) + 1$$

#### 2. LDA算法的基本假设是什么？

LDA即Latent Dirichlet Allocation，隐含狄利克雷分布，其基本假设是文档主题以狄利克雷分布为先验分布，而每个主题中词的先验分布也是狄利克雷分布。

#### 3. 在TextRank算法中构建图的权重是如何得到的？

TextRank首先使用滑窗方式获取词之间的连边，连边权重通过计算两个词的相似性（距离）得出。在图构建完毕后，开始进行迭代，即对每个节点取其相连节点的权重，使用$WS(v_i)':=(1-d) + d \times \sum{v_j\in In(v_i)}\frac{w_{ij}}{\sum{v_k}\in Out(v_j)w_{jk}}WS(v_j)$进行计算，不断迭代更新各个节点的权重，直至收敛。

#### 4. 什么是命名实体识别？ 有什么应用场景？

命名实体识别即Named Entity Recognition(NER)，是信息提取的一个子任务，旨在从大段的文本数据中提取出命名实体并进行分类，例如人名、地名、组织、货币、金额、日期等等。  
NER的应用范围非常广泛，例如文本摘要，简历关键词提取，通过抽取商品介绍内容进行更精准的分类和推荐，都可以使用NER或与之类似的思想来完成。

#### 5.NLP主要有哪几类任务 ？

NLP任务多种多样，常见的NLP任务如：
 - 语音识别
 - 机器翻译
 - 自动问答客服
 - 文本分类
 - 自动摘要
 - 文本/语音生成
 - 关键词提取
 - 分词
 - 知识图谱

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

导入工具包

In [6]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

from collections import defaultdict
from gensim.models import word2vec
import re
import jieba

导入数据

In [81]:
data = pd.read_csv("新华社数据.csv", encoding="gb18030")

随机提取100条数据

In [89]:
sub_data = data.sample(n=100, random_state=80)

In [90]:
sub_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 42742 to 34304
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       100 non-null    int64 
 1   author   86 non-null     object
 2   source   100 non-null    object
 3   content  100 non-null    object
 4   feature  100 non-null    object
 5   title    100 non-null    object
 6   url      100 non-null    object
dtypes: int64(1), object(6)
memory usage: 6.2+ KB


分词

In [84]:
# 读取停止词
with open("./stopwords.txt", encoding="utf-8-sig") as f:
    stopwords = [line.strip() for line in f.readlines()]

In [109]:
def cut_it(content, stopwords=None):
    # 去除标点及特殊字符
    content = re.sub('[!；：．·、…【】《》“”‘’！？"#$%&％\'?@，。〔〕［］（）()*+,\\./:;<=>＋×／'
                      '①↑↓★▌▲●℃[\\]^_`{|}\s\\\\n]+',
                      "",
                      content)
    words = jieba.lcut(content)
    # 过滤停用词
    if stopwords:
        filter_words = [word for word in words if word not in stopwords]
    else:
        filter_words = words
    return " ".join(filter_words)

In [104]:
test = "新华社照片利马年月日\\n体育趣味狗拉松\n月日一名男子带领宠物犬参赛\n当日秘鲁首都利马一场宠物犬参加马拉松赛比赛中宠物主人带领宠物犬千米道路奔跑\n新华社发路易斯卡马乔摄"
cut_it(test, stopwords)

'新华社 照片 利马 年月日 \\ n 体育 趣味 狗拉松 月 日 一名 男子 带领 宠物犬 参赛 当日 秘鲁 首都 利马 一场 宠物犬 参加 马拉松赛 比赛 中 宠物 主人 带领 宠物犬 千米 道路 奔跑 新华社 发 路易斯 卡马乔 摄'

In [110]:
sub_data['content'] = sub_data['content'].apply(cut_it, stopwords=stopwords)

In [111]:
contents = [content.split(" ") for content in sub_data['content']]

训练词向量

In [112]:
wv_model = word2vec.Word2Vec(contents, size=50, window=5, min_count=1, iter=10, workers=4)

构建节点和图类

In [124]:
class node:
    '''
    节点
    '''
    def __init__(self, word=None, weight=0):
        '''
        初始化节点信息
        --------------
        word: 节点包含的词
        weight: 初始权重
        '''
        self.word = word
        self.weight = weight
        self.edges = defaultdict(float)
        self.out_edge_total = 0
    
    def __getitem__(self, word2):
        '''
        获取到word2的连边权重
        -------------------
        word2: 获取连边权重的目标词
        '''
        return self.edges[word2]
        
    def calc_similarity(self, word2, method='cosine'):
        '''
        计算相似度
        -----------
        word2: 节点所连的词
        method: 计算方式
        '''
        similarity = 0
        if method == 'cosine':
            similarity = cosine(wv_model.wv[self.word], wv_model.wv[word2])
        elif method == 'L2':
            similarity = np.sqrt(np.sum([pow(x, 2) for x in (wv_model.wv[self.word] - wv_model.wv[word2])]))
        else:
            similarity = np.abs(wv_model.wv[self.word] - wv_model.wv[word2])
            if len(similarity) > 1:
                similarity = sum(similarity)
        return similarity
        
    def add_edge_to(self, w2, similarity=None, include_co_occur=False):
        '''
        添加连边
        ---------
        w2: 需要连接的词或节点
        similarity: 词语相似度
        '''
        if similarity == None:
            if not include_co_occur and w2.word in self.edges:
                return
            # 等于None时进行计算，并为所连词添加连边
            similarity = self.calc_similarity(w2.word)
            self.edges[w2.word] += similarity
            w2.add_edge_to(self.word, similarity, include_co_occur)
        else:
            if not include_co_occur and w2 in self.edges:
                return
            # 不等于None时直接记录连边权重
            self.edges[w2] += similarity
        # 更新连边权重总和
        self.out_edge_total += similarity
        
    def remove_edge_to(self, w2):
        '''
        移除连边
        --------
        w2: 需要断开连接的词或节点
        '''
        if type(w2) == node:
            similarity = self.edges.pop(w2.word)
            w2.remove_edge_to(self.word)
        else:
            similarity = self.edges.pop(w2)
        # 更新连边权重总和
        self.out_edge_total -= similarity
        
    def is_linked_to(self, w2):
        '''
        判断是否与目标词相连
        --------------------
        w2: 目标词
        '''
        return self.edges[w2] != 0
    
    def update(self, graph, converged_value=1e-7, d=0.85):
        '''
        更新节点权重
        -------------
        graph: 图的引用
        converged_value: 判断是否收敛的依据
        '''
        converged = True
        if not self.edges:
            return converged
        new_weight = 0
        for word2, similarity in self.edges.items():
            opposite_node = graph[word2]
            if opposite_node.word != None:
                new_weight += opposite_node.weight * (similarity/opposite_node.out_edge_total)
        new_weight = (1 - d) + d * new_weight
        if abs(new_weight - self.weight) > 1e-7:
            converged = False
        self.weight = new_weight
        return converged
    

In [125]:
class graph:
    '''
    图
    '''
    def __init__(self, contents, d=0.85):
        '''
        初始化图信息
        -------
        contents: 分词后的句子列表
        '''
        if not contents:
            return
        # 初始化
        self.d = d
        self.nodes = defaultdict(node)
        self.contents = contents
        seen_words = set()
        # 生成节点
        for content in contents:
            for word in content:
                # 跳过已记录的词
                if word in seen_words:
                    continue
                self.nodes[word] = node(word, 0)
                seen_words.add(word)
                
        # 更新节点初始权重
        n = len(seen_words)
        del seen_words
        init_weight = 1/n
        for node_ in self.nodes.values():
            node_.weight = init_weight
            
    def __getitem__(self, word):
        '''
        获取word对应节点
        -------------------
        word: 目标词
        '''
        return self.nodes[word]
    
    def build_edges(self, window_size=5, include_co_occur=False):
        '''
        使用滑动窗口，为所有节点建立连边
        --------------------------------
        window_size: 窗口大小
        include_co_occur: 是否考虑词语共现次数
        '''
        if not self.contents:
            raise Exception("图已建立或没有输入正确的contents内容！")
        n = window_size >> 1
        # 遍历句子
        for content in self.contents:
            # 遍历词语
            for i in range(len(content)):
                w1 = content[i]
                sub_content = []
                if i < n:
                    sub_content = content[:i] + content[i+1:i+n+1]
                elif i + n >= len(content):
                    sub_content = content[i-n:i] + content[i+1:]
                else:
                    sub_content = content[i-n:i] + content[i+1:i+n+1]
                for w2 in sub_content:
                    self.nodes[w1].add_edge_to(self.nodes[w2], include_co_occur=include_co_occur)
        
        # 清除
        self.contents = None
    
    def start_iterations(self, max_iter=100, converged_value=1e-7):
        '''
        开始迭代更新节点参数
        --------------------
        max_iter: 最大迭代次数
        converged_value: 判断是否收敛的依据
        '''
        # 初始化flag和迭代计数
        converged = False
        count = 0
        # 循环直到收敛
        while not converged:
            converged = True
            # 遍历节点，更新权重，记录权重参数是否变更
            for node_ in self.nodes.values():
                converged &= node_.update(self, converged_value, self.d)
            # 增加计数
            count += 1
            # 判断达到最大迭代次数，退出
            if count >= max_iter:
                break
        # 返回最终结果是否收敛
        return converged, count
    
    def most_important(self, text, n=5):
        '''
        获取文本中权重最高的n个词
        -------------------------
        text: 分词后的文本
        n: 返回数目
        '''
        if type(text) == str:
            text = text.split(" ")
        result = {(word, self[word].weight) for word in text}
        result = sorted(result, key=lambda x: x[1], reverse=True)
        return result[:n]

In [150]:
def rankMultiText(contents):
    G = graph(contents)
    G.build_edges()
    print(G.start_iterations())
    return G

In [151]:
def rankSingleText(content):
    G = graph([content])
    G.build_edges()
    print(G.start_iterations())
    return G

In [152]:
def rank(G, text, n=20):
    print("".join(text))
    return G.most_important(text, n=n)

In [164]:
G = rankMultiText(contents)

(True, 60)


In [173]:
sorted(list(G["新华社"].edges.items()), key=lambda x:x[1], reverse=True)[:20]

[('雷摄', 1.2433893978595734),
 ('漯河', 1.2090365439653397),
 ('品酒', 1.1985060721635818),
 ('拉斯', 1.1939423978328705),
 ('福州', 1.19217748939991),
 ('勒斯', 1.1817264407873154),
 ('柏林', 1.17827570438385),
 ('青岛', 1.1651315689086914),
 ('冲凉', 1.1604227125644684),
 ('爱不释手', 1.1482814699411392),
 ('奔跑', 1.144282504916191),
 ('对阵', 1.1440399289131165),
 ('快讯', 1.140726163983345),
 ('南昌', 1.1388383507728577),
 ('利兹联', 1.1336669921875),
 ('大本钟', 1.1304892748594284),
 ('赫尔辛基', 1.124107114970684),
 ('太原', 1.1234295144677162),
 ('庞兴雷', 1.1233981475234032),
 ('平摄', 1.1229473128914833)]

In [174]:
rank(G, contents[1])

新华社照片利马2017年4月24日体育2趣味狗拉松4月23日一名男子带领宠物犬参赛当日秘鲁首都利马一场宠物犬参加马拉松赛比赛中宠物主人带领宠物犬15千米道路奔跑新华社发路易斯卡马乔摄


[('新华社', 26.658834014239588),
 ('年', 20.239226631534653),
 ('日', 18.702198049253877),
 ('月', 16.202038665004682),
 ('中', 16.024042785365843),
 ('4', 9.495607263515184),
 ('2017', 7.885232289863415),
 ('体育', 5.670393356770726),
 ('当日', 5.396655531806085),
 ('照片', 4.914455612849356),
 ('比赛', 4.848614888694893),
 ('摄', 4.711916351525653),
 ('2', 3.494097564568777),
 ('带领', 3.4539619680179467),
 ('参加', 2.6060575592722754),
 ('首都', 2.4794486862646785),
 ('发', 2.3233665665303644),
 ('一名', 2.2352885975541334),
 ('15', 1.9199825904869372),
 ('道路', 1.8478514323789126)]

In [175]:
rank(G, contents[2])

一以深化放管服改革进一步铲除滋生腐败土壤减权限权是预防腐败釜底抽薪之策政府一场深刻革命政府部门壮士断腕勇气扎实举措推进放管服改革企业群众广开便利门政府履职廉洁高效清单管理推动减权规范用权放管服改革减权限权数量成效实践中发现领域审批事项保留审批事项名义一项实际上细分小项成权力套娃部门自行设置类事项减了名目增加特别审批事项相关不合时宜精简空间很大审批抬高制度性交易成本寻租腐败简政放权改革持续深化限度权力压减到位规范铲除滋生腐败土壤实行清单管理制度深化放管服改革抓手建立规范政府权力责任总台账所有权责事项详尽列入清单明明白白社会公布清单之内政府部门履职尽责清单之外禁止擅自设权扩权切实权力置于制度框架规范运行抓紧制定国务院部门权力责任清单清单制定减权过程减则减除涉及公共利益事项外行政审批事项原则上依法程序取消市场准入负面清单试点加快制定工商登记前置审批事项清单企业设立经营许可清单确需保留实行多证合一证照联办证照分离试点推动实体经济转型升级涉及工业产品审批许可地方同志工业生产领域审批产品涉及十大类地方化工产品配方制造业工艺流程审批配方工艺企业商业秘密审批做企业耗费时间贻误市场先机加大知识产权外泄风险影响企业创新积极性导致寻租清理大幅压减工业产品生产许可证确需保留制定清单严格管理企业创新清障松绑政府性基金行政事业性收费中介机构行业协会商会涉企收费实行清单管理各类清单公开晾晒社会监督腐败藏身地二要创新事中事后监管保障廉洁执法放权减权政府部门精力转事中事后监管两年推行双随机公开监管随机抽查加执法公开监管对象头上利剑高悬始终感到监管无形压力心存侥幸任性检查和执法过程中人为干扰减少监管部门寻租腐败双随机公开监管全覆盖加快推进跨部门联合检查有利于减轻企业负担检查部门相互监督信用监管联合惩戒建国家企业信用信息归集系统推动部门间地区间涉企信息交换共享提高监管效能推进综合执法改革规范公正文明执法重复执法多头执法粗暴执法减少企业干扰智能监管数据监管监管全过程留痕权力滥用三要优化服务政务清廉推行互联网政务服务加快国务院部门地方政府信息系统互联互通全国统一政务服务平台国务院部门信息平台互不联通措施打通信息孤岛提高企业群众办事便利性效率监督减少办事人员吃拿卡要机会便民高效廉洁政风进一步提升实体政务大厅服务能力水平加快网上服务平台融合发展网上办上网实行一号申请一窗受理一网通办减证便民专项证明手续取消取消合

[('中', 16.024042785365843),
 ('发展', 14.972746995946505),
 ('企业', 9.716790674036147),
 ('政府', 8.553104338116862),
 ('5', 8.54035031621638),
 ('国家', 8.333450671861629),
 ('群众', 7.714644556577309),
 ('新', 6.976208682396747),
 ('创新', 6.6478824799576905),
 ('资金', 6.393837908616948),
 ('经济', 6.072268637466087),
 ('市场', 5.3640820618644796),
 ('推动', 5.234606313645496),
 ('项目', 4.823531190120483),
 ('时间', 4.8049392339402415),
 ('相关', 4.785162524177267),
 ('平台', 4.592819936006553),
 ('生产', 4.526150706413196),
 ('管理', 4.312600823954643),
 ('部门', 4.233017202802898)]

In [176]:
rank(G, contents[3])

新华社照片巴黎2017年6月7日体育2网球——法网巴辛斯基晋级半决赛6月6日巴辛斯基比赛中回球当日法国巴黎2017法国网球公开赛女子单打四分之一决赛中瑞士选手巴辛斯基以20战胜法国选手梅拉德诺维奇晋级半决赛新华社记者陈益宸摄


[('新华社', 26.658834014239588),
 ('年', 20.239226631534653),
 ('日', 18.702198049253877),
 ('记者', 17.811202525006497),
 ('月', 16.202038665004682),
 ('中', 16.024042785365843),
 ('—', 9.750064989967399),
 ('2017', 7.885232289863415),
 ('体育', 5.670393356770726),
 ('当日', 5.396655531806085),
 ('选手', 5.10530770656837),
 ('照片', 4.914455612849356),
 ('比赛', 4.848614888694893),
 ('摄', 4.711916351525653),
 ('7', 3.983290104986436),
 ('战胜', 3.8151792213713622),
 ('2', 3.494097564568777),
 ('6', 3.044742106964373),
 ('法国', 3.02216743601831),
 ('晋级', 2.710344721637468)]

In [177]:
G = rankSingleText(contents[1])

(True, 52)


In [178]:
sorted(list(G["新华社"].edges.items()), key=lambda x:x[1], reverse=True)

[('奔跑', 1.144282504916191),
 ('利马', 1.0305679179728031),
 ('路易斯', 0.869931772351265),
 ('道路', 0.7804607003927231),
 ('照片', 0.7421922981739044),
 ('发', 0.5765703320503235)]

In [179]:
rank(G, contents[1])

新华社照片利马2017年4月24日体育2趣味狗拉松4月23日一名男子带领宠物犬参赛当日秘鲁首都利马一场宠物犬参加马拉松赛比赛中宠物主人带领宠物犬15千米道路奔跑新华社发路易斯卡马乔摄


[('宠物犬', 2.466519434019129),
 ('利马', 2.122733764813277),
 ('带领', 1.4200025883446132),
 ('新华社', 1.3772103176738515),
 ('趣味', 1.198522744880457),
 ('狗拉松', 1.191482100905572),
 ('路易斯', 1.176636989593093),
 ('4', 1.1086256599420334),
 ('奔跑', 1.0935885022231164),
 ('马拉松赛', 1.078783708872828),
 ('发', 1.0294562352635714),
 ('男子', 1.0260332846697315),
 ('秘鲁', 1.022790098457936),
 ('千米', 0.999306787977641),
 ('参赛', 0.9806810896000895),
 ('日', 0.971867107197245),
 ('宠物', 0.9705771996222836),
 ('主人', 0.970456309474833),
 ('首都', 0.9273991291362361),
 ('一名', 0.9267133081468271)]

In [180]:
G = rankSingleText(contents[2])
rank(G, contents[2])

(True, 55)
一以深化放管服改革进一步铲除滋生腐败土壤减权限权是预防腐败釜底抽薪之策政府一场深刻革命政府部门壮士断腕勇气扎实举措推进放管服改革企业群众广开便利门政府履职廉洁高效清单管理推动减权规范用权放管服改革减权限权数量成效实践中发现领域审批事项保留审批事项名义一项实际上细分小项成权力套娃部门自行设置类事项减了名目增加特别审批事项相关不合时宜精简空间很大审批抬高制度性交易成本寻租腐败简政放权改革持续深化限度权力压减到位规范铲除滋生腐败土壤实行清单管理制度深化放管服改革抓手建立规范政府权力责任总台账所有权责事项详尽列入清单明明白白社会公布清单之内政府部门履职尽责清单之外禁止擅自设权扩权切实权力置于制度框架规范运行抓紧制定国务院部门权力责任清单清单制定减权过程减则减除涉及公共利益事项外行政审批事项原则上依法程序取消市场准入负面清单试点加快制定工商登记前置审批事项清单企业设立经营许可清单确需保留实行多证合一证照联办证照分离试点推动实体经济转型升级涉及工业产品审批许可地方同志工业生产领域审批产品涉及十大类地方化工产品配方制造业工艺流程审批配方工艺企业商业秘密审批做企业耗费时间贻误市场先机加大知识产权外泄风险影响企业创新积极性导致寻租清理大幅压减工业产品生产许可证确需保留制定清单严格管理企业创新清障松绑政府性基金行政事业性收费中介机构行业协会商会涉企收费实行清单管理各类清单公开晾晒社会监督腐败藏身地二要创新事中事后监管保障廉洁执法放权减权政府部门精力转事中事后监管两年推行双随机公开监管随机抽查加执法公开监管对象头上利剑高悬始终感到监管无形压力心存侥幸任性检查和执法过程中人为干扰减少监管部门寻租腐败双随机公开监管全覆盖加快推进跨部门联合检查有利于减轻企业负担检查部门相互监督信用监管联合惩戒建国家企业信用信息归集系统推动部门间地区间涉企信息交换共享提高监管效能推进综合执法改革规范公正文明执法重复执法多头执法粗暴执法减少企业干扰智能监管数据监管监管全过程留痕权力滥用三要优化服务政务清廉推行互联网政务服务加快国务院部门地方政府信息系统互联互通全国统一政务服务平台国务院部门信息平台互不联通措施打通信息孤岛提高企业群众办事便利性效率监督减少办事人员吃拿卡要机会便民高效廉洁政风进一步提升实体政务大厅服务能力水平加快网上服务平台融合发展网上办上网实行一号申请一窗受理一网通办减证便民

[('资金', 5.362874586432887),
 ('清单', 4.508112555449612),
 ('监管', 4.397294839292606),
 ('政府', 4.2525421431204045),
 ('审批', 4.068895802098536),
 ('加快', 3.860872336356502),
 ('部门', 3.4014858016634237),
 ('企业', 3.335006802819762),
 ('事项', 3.2940794885755844),
 ('规范', 3.202527453859185),
 ('改革', 3.1781412389559476),
 ('腐败', 3.1675287961604885),
 ('地方', 3.078274429583967),
 ('监督', 3.0286738819120984),
 ('权力', 2.88833329271991),
 ('国有资产', 2.7356686358884983),
 ('预算', 2.506709025118639),
 ('实行', 2.408143598505686),
 ('风险', 2.394682516475521),
 ('整合', 2.380964805397552)]

感觉TextRank的效果一般，结果有些类似TF，出现频率越高的词语，最后的权重就越大，在多篇文章同时处理时更为明显，基本每篇文章最重要的词都是新华社…如果不去除停用词和标点的话，排在前面的就全是标点符号和“的”之类的词了…

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。

### 导入工具包

In [441]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import time
import pickle

### 文本处理

In [502]:
def read_file(path, sentence_end="\n", encoding="utf-8"):
    with open(path, "r", encoding=encoding) as f:
        content = f.read().split(sentence_end)
    return content

In [517]:
def cut(texts):
    '''
    分词
    -----------
    texts: 句子列表
    '''
    return [jieba.lcut(text.replace(u'\u3000',u'')) for text in texts]

In [578]:
def init_tokenizer(texts):
    '''
    初始化tokenizer
    ------------------
    texts: 已分词的句子列表
    '''
    # 添加UNK标识
    texts_ = texts + [["<UNK>","<NONE>"]]
    # 实例化
    tokenizer = Tokenizer(filters='#$%&+-/<=>@[\\]^`{|}~\t\n')
    # 生成倒排表
    tokenizer.fit_on_texts(texts_)
    return tokenizer

In [87]:
def tokenize(texts, tokenizer):
    '''
    将句子转化为token列表
    ----------------------
    texts: 已分词的句子列表
    tokenizer: token转换器
    '''
    # 未知词语转换为UNK标识
    texts = [[word if word in tokenizer.word_index else "<UNK>" for word in text] for text in texts]
    # 将词语转为token
    sequence = tokenizer.texts_to_sequences(texts)
    # padding
    padded = pad_sequences(sequence, padding='post')
    return padded

### 模型

In [579]:
class LanguageModel(Model):
    def __init__(self, vocab_size, embedding_dim, batch_size, hidden_units):
        '''
        初始化模型
        ---------------------
        vocab_size: 词库大小
        embedding_dim: 词嵌入维度
        batch_size: 批次大小
        hidden_units: GRU_1,GRU_2,和FC_1三层的隐层神经元数目列表
        '''
        assert len(hidden_units) == 3
        super(LanguageModel, self).__init__()
        self.batch_size = batch_size
        self.hidden_units = hidden_units
        # 嵌入层
        self.embedding = layers.Embedding(vocab_size, embedding_dim)
        # GRU_1
        self.gru_1 = layers.GRU(hidden_units[0], return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        # GRU_2
        self.gru_2 = layers.GRU(hidden_units[1], return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        # FC_1
        self.dense_1 = layers.Dense(hidden_units[2])
        # FC_2
        self.dense_2 = layers.Dense(vocab_size)
    
    def call(self, x, hidden):
        '''
        调用模型
        -------------------------
        x: 数据,(batch_size, sequence_length)
        hidden: GRU_1和GRU_2的隐层状态列表
        '''
        if len(x.shape) <= 1:
            x = tf.reshape(x, (-1,1))
        # embedding => (batch_size, sequence_length, embedding_dim)
        x = self.embedding(x)
        # gru => output(batch_size, sequence_length, hidden_units[0]), state(batch_size, hidden_units[0])
        output, state = self.gru_1(x, hidden[0])
        # gru_2 => output(batch_size, sequence_length, hidden_units[1]), state(batch_size, hidden_units[1])
        output, state_2 = self.gru_2(output, hidden[1])
        # reshape => (batch_size*sequence_length, hidden_units[1])
        output = tf.reshape(output, (-1, output.shape[2]))
        # fc_1 => (batch_size, hidden_units[2])
        output = self.dense_1(output)
        # fc_2 => (batch_size, vocab_size)
        output = self.dense_2(output)
        # 返回结果
        return output, state, state_2
    
    def initialize_hidden_state(self, batch_size = None):
        '''
        使用全零向量初始化GRU隐层状态
        '''
        if not batch_size:
            batch_size = self.batch_size
        return [tf.zeros((batch_size, self.hidden_units[0])), tf.zeros((batch_size, self.hidden_units[1]))]
        

In [631]:
import pdb

In [667]:
class AutoWriter:
    def __init__(self, padded_texts):
        '''
        初始化，建立数据集
        -------------------
        padded_texts: 分词，转为token并padding后的文本列表
        '''
        self.dataset = tf.data.Dataset.from_tensor_slices(padded_texts)
        self.data_length = len(padded_texts)
        
    def init_model(self, vocab_size, embedding_dim, batch_size, hidden_units, tokenizer):
        '''
        初始化模型
        ------------------
        vocab_size: 词库大小
        embedding_dim: 词嵌入维度
        batch_size: 批次大小
        hidden_units: GRU_1,GRU_2,和FC_1三层的隐层神经元数目列表
        tokenizer: token转换器，包含词语-索引的倒排表
        '''
        self.language_model = LanguageModel(vocab_size, embedding_dim, batch_size, hidden_units)
        self.tokenizer = tokenizer
        self.set_data_batch(batch_size)
        
    def load_weights(self, path, token_name=None, include_tokenizer=False):
        '''
        读取预训练的模型权重
        --------------------
        path: 权重路径
        token_name: token转换器的路径
        include_tokenizer: 是否需要同时读取token转换器
        '''
        self.language_model.load_weights(path)
        if include_tokenizer:
            if not token_name:
                token_name = "tokenizer.pickle"
            with open(token_name, 'rb') as f:
                self.tokenizer = pickle.load(f)
        print("Model weights loaded")
        
    def save_model(self, path, token_name=None, include_tokenizer=True):
        '''
        保存训练好的模型权重
        -----------------
        path: 保存模型权重的路径
        token_name: token转换器的路径
        include_tokenizer: 是否需要同时保存token转换器
        '''
        self.language_model.save_weights(path)
        if include_tokenizer:
            if not token_name:
                token_name = "tokenizer.pickle"
            with open(token_name, 'wb') as f:
                pickle.dump(self.tokenizer, f)
        print("Model weights saved to:", path)
        
    def set_data_batch(self, batch_size):
        '''
        设置batch size
        ---------------
        batch_size: 批次大小
        '''
        self.steps_per_epoch = self.data_length // batch_size
        self.dataset = self.dataset.batch(batch_size, drop_remainder=True)
    
    def calc_loss(self, label, pred):
        '''
        计算损失
        ----------
        label: 标签
        pred: 预测结果
        '''
        # 使用损失函数，获取loss
        loss = self.loss_function(label, pred)
        # 对label为0的部分生成mask，屏蔽padding的部分
        mask = tf.math.logical_not(tf.math.equal(label, 0))
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        
        return tf.reduce_mean(loss)
    
    def train_step(self, inputs, hidden=None):
        '''
        单步训练
        ----------
        inputs: 一个批次的输入内容，(batch_size, sequence_length)
        hidden: GRU_1和GRU_2的隐层状态
        '''
        # 初始化隐层状态
        if not hidden:
            hidden = self.language_model.initialize_hidden_state()
        # 初始化loss
        loss = 0
        # 设置tape
        with tf.GradientTape() as tape:
            # 遍历所有时间步
            for t in range(1, inputs.shape[1]):
                # 调用模型获取预测输出和更新后的GRU隐层状态
                pred, hidden[0], hidden[1] = self.language_model(inputs[:, t-1], hidden)
                # 调用损失计算，累加损失
                loss += self.calc_loss(inputs[:, t], pred)
        # 用loss除以sequence_length(time steps)求均值
        batch_loss = loss / int(inputs.shape[1])
        # 取出模型可训练参数
        trainables = self.language_model.trainable_variables
        # 计算梯度
        gradients = tape.gradient(loss, trainables)
        # 更新梯度
        self.optimizer.apply_gradients(zip(gradients, trainables))
        return batch_loss
        
    def train(self, loss_function=None, epochs=10, learning_rate=1e-3):
        '''
        训练模型
        -----------------
        loss_function: 损失函数
        epochs: 训练轮数
        learning_rate: 学习率
        '''
        # 设置损失函数
        if not loss_function:
            loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        self.loss_function = loss_function
        # 设置优化器
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)
        for epoch in range(epochs):
            # 记录开始时间
            start = time.time()
            loss = 0
            # 分batch取出数据
            for batch, inputs in enumerate(self.dataset.take(self.steps_per_epoch)):
                # 训练一步
                batch_loss = self.train_step(inputs)
                # 累计loss
                loss += batch_loss
                # 每100个batch打印输出
                if batch % 100 == 0:
                    print("Epoch {} Batch {} Loss {:.4f}".format(epoch + 1, batch, batch_loss.numpy()))
            # 打印epoch信息，并调用eval_检查模型输出
            print("Epoch {} finished, Loss: {:.4f}, Time cost: {} seconds\n".format(epoch+1, loss / self.steps_per_epoch, time.time()-start))
            print(self.eval_())
            
    def eval_(self, sentences=2):
        '''
        检查模型输出，便于评估
        -----------------------
        sentences: 输出句子数目
        '''
        index = int(np.random.random() * (self.steps_per_epoch - 1)) + 1
        input_ = list(self.dataset.take(index))[-1][0]
        return self.write(input_, sentences=sentences)
    
    def write(self, pre_text, sentence_end="。", sentences=5, max_words=50):
        '''
        续写文章
        ---------
        pre_text: 前文内容
        sentence_end: 句子结束标识
        sentences: 需要生成的句子数目
        max_words: 每个句子最大词数
        '''
        # 处理不存在于数据中的句子结束标识
        if sentence_end not in self.tokenizer.word_index:
            raise ValueError("{} is not a known word!".format(sentence_end))
        
        # 是否需要将输入内容转换为词语
        need_mapping = True
        # 初始化输出结果
        output = ""
        # 如果输入前文是文本，对其进行分词并转为token，同时记录前文内容
        if type(pre_text) == str:
            output = pre_text
            need_mapping = False
            pre_text = cut(pre_text)
            pre_text = tokenize(pre_text, self.tokenizer)
            
        # 转为tensor
        pre_text = tf.constant(pre_text)
        # 初始化隐层状态
        hidden = self.language_model.initialize_hidden_state(1)
        # 依次输入前文
        for word in pre_text:
            # 将index转换为词语记录
            if need_mapping:
                try:
                    output += self.tokenizer.index_word[word.numpy()]
                except:
                    output += ""
            # 调用模型
            next_, hidden[0], hidden[1] = self.language_model(word, hidden)
        # 句子计数
        sentence_count = 0
        # 单句词语计数
        word_count = 0
        # 获取模型最后的输出index
        next_word_index = np.argmax(next_)
        output += ">>>new>>>"
        # 循环直到输出句子数目达到要求
        while sentence_count < sentences:
            # 调用模型
            next_, hidden[0], hidden[1] = self.language_model(next_word_index, hidden)
            # 获取输出index
            next_word_index = np.argmax(next_)
            # 获取对应词语并记录
            word = self.tokenizer.index_word[next_word_index]
            output += word
            word_count += 1
            # 如果输出了句末标识，则增加句子计数，重置单句词语计数
            if word == sentence_end:
                word_count = 0
                sentence_count += 1
            else:
                # 否则增加单句词语计数，并判断是否达到上限
                # 达到上限则强制添加句末标识，结束句子，并将标识作为下一次模型调用的输入
                word_count += 1
                if word_count == max_words:
                    output += "。"
                    word_count = 0
                    sentence_count += 1
                    next_ = self.tokenizer.word_index[sentence_end]
                    
        return output    
        

In [581]:
content = read_file("./text.txt",encoding="gb18030")

In [582]:
len(content)

15494

In [583]:
content[10]

'\u3000\u3000在黑暗的空间艰难跋涉的我眼前一道仿佛天堂的门凭空开启。'

In [584]:
content = cut(content)

In [585]:
content[10]

['在',
 '黑暗',
 '的',
 '空间',
 '艰难',
 '跋涉',
 '的',
 '我',
 '眼前',
 '一道',
 '仿佛',
 '天堂',
 '的',
 '门',
 '凭空',
 '开启',
 '。']

In [586]:
tokenizer = init_tokenizer(content)

In [587]:
content = tokenize(content, tokenizer)

In [588]:
content[10]

array([  12,  507,    2,  699, 2859, 3478,    2,    4,  274,  611,  349,
       1683,    2, 1810, 1460, 1388,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [589]:
len(content[10])

195

In [590]:
len(tokenizer.word_index)

24530

In [598]:
embedding_dim = 64
batch_size = 16
hidden_units = [128, 64, 32] # gru_1, gru_2, fc_1

In [591]:
writer = AutoWriter(content)

In [592]:
writer.init_model(len(tokenizer.word_index), embedding_dim, batch_size, hidden_units, tokenizer)

In [542]:
writer.load_weights("language_model", True)

Model weights loaded


In [593]:
writer.write(content[3][:50], "。", 2)

'>>>new>>>之极之极化解获益匪浅抗下打了个打了个一箭抗下精嘛2289惊天手伸篮球法相场内外2一分钱很久没很久没列游行中探游行找点。拣到2289崩溃排除不象话气极败坏玩吧玩吧2票人场内外一哄而散逸哥打紧时机未到没人能援军战战兢兢战战兢兢我地动静动静求援失忆失忆。'

In [594]:
writer.train(epochs=5)

Epoch 1 Batch 0 Loss 0.4697
Epoch 1 Batch 100 Loss 1.3704
Epoch 1 Batch 200 Loss 1.2771
Epoch 1 Batch 300 Loss 1.1158
Epoch 1 Batch 400 Loss 1.3315
Epoch 1 Batch 500 Loss 1.9137
Epoch 1 Batch 600 Loss 1.2137
Epoch 1 Batch 700 Loss 0.9326
Epoch 1 Batch 800 Loss 1.5752
Epoch 1 Batch 900 Loss 0.8377
Epoch 1 finished, Loss: 1.3130, Time cost: 6205.0008001327515 seconds

妈的，是可忍!孰不可忍，刚到现场我就有了想挂人的冲动，忍到现在无非是怕先动手让对方知道名字引起不必要的麻烦。而楚时月现身之后又立刻隐起来不出来制止，没有脸皮是第一，有心想看看我的本事应该是第二。>>>new>>>，，，，，，，，，，，，，，，，，，，，，，，，，。，，，，，，，，，，，，，，，，，，，，，，，，，。
Epoch 2 Batch 0 Loss 0.3312
Epoch 2 Batch 100 Loss 1.3056
Epoch 2 Batch 200 Loss 1.2103
Epoch 2 Batch 300 Loss 1.0633
Epoch 2 Batch 400 Loss 1.0640
Epoch 2 Batch 500 Loss 1.5334
Epoch 2 Batch 600 Loss 1.0919
Epoch 2 Batch 700 Loss 0.8556
Epoch 2 Batch 800 Loss 1.4729
Epoch 2 Batch 900 Loss 0.7768
Epoch 2 finished, Loss: 1.1714, Time cost: 6386.233181238174 seconds

搂着怀里的叶子，我们慢慢前进着，眼前除了那些高低不一却都需要几人合抱的树桩之外，没有任何建筑，也没有任何花花草草。如同梅花桩一般的迷宫……>>>new>>>我，我是是的的的的的的的的的的时候，我的的的的的的的

In [597]:
writer.save_model("language_model")

Model weights saved to: language_model


In [596]:
writer.train(epochs=5)

Epoch 1 Batch 0 Loss 0.2206
Epoch 1 Batch 100 Loss 1.0573
Epoch 1 Batch 200 Loss 0.9474
Epoch 1 Batch 300 Loss 0.9002
Epoch 1 Batch 400 Loss 0.6595
Epoch 1 Batch 500 Loss 1.1540
Epoch 1 Batch 600 Loss 0.8748
Epoch 1 Batch 700 Loss 0.7384
Epoch 1 Batch 800 Loss 1.2632
Epoch 1 Batch 900 Loss 0.6303
Epoch 1 finished, Loss: 0.9197, Time cost: 6814.959317445755 seconds

组了两个战士之后跟着他们出了泰西城，本来还想组一个术士的，无奈这年头术士相当稀少，只好作罢，好在开设了货币兑换之后药剂的价格还没有上涨，靠喝药抗显然没有多大问题。>>>new>>>而且，我也是一个<unk>的。而我，我也是一个一个<unk>的。
Epoch 2 Batch 0 Loss 0.2083
Epoch 2 Batch 100 Loss 1.0063
Epoch 2 Batch 200 Loss 0.8835
Epoch 2 Batch 300 Loss 0.8517
Epoch 2 Batch 400 Loss 0.6185
Epoch 2 Batch 500 Loss 1.1103
Epoch 2 Batch 600 Loss 0.8386
Epoch 2 Batch 700 Loss 0.7163
Epoch 2 Batch 800 Loss 1.2132
Epoch 2 Batch 900 Loss 0.5939
Epoch 2 finished, Loss: 0.8773, Time cost: 6738.136963844299 seconds

“……请您实现契约的诺言，赐予我无穷的力量……”>>>new>>>而我，我也是一个一个一个一个一个工作室的时候，我也是一个一个的。而我，我也是一个一个的时候，我也是一个一个的。
Epoch 3 Batch 0 Loss 0.2041
Epoch 3 Batch 100 Loss 0

In [610]:
content = read_file("./short_text.txt",encoding="utf-8")

In [611]:
content = cut(content)

In [612]:
tokenizer_2 = init_tokenizer(content)

In [613]:
content = tokenize(content, tokenizer_2)

In [622]:
embedding_dim = 64
batch_size = 1
hidden_units = [128, 64, 32] # gru_1, gru_2, fc_1

In [668]:
writer = AutoWriter(content)

In [669]:
writer.init_model(len(tokenizer_2.word_index), embedding_dim, batch_size, hidden_units, tokenizer_2)

In [670]:
writer.write(content[3][:50], "。", 2)

'旧时好多有传承的人家，吃饭有个规矩：一桌子琳琅佳肴前，先吃三口白饭。长辈一代代教诲：第一口必须先吃饭，而绝不能没吃饭就夹菜。>>>new>>>压个压个压个面食压个面食面食上一餐饭象征性总养生养生本自本自自这个多本自本自本自本自前前供养。占据占据<unk>传承教诲白米饭<unk>琳琅琳琅菜恬淡<unk>陪衬陪衬创伤创伤几口吃饭吃饭。'

In [647]:
writer.train(epochs=50)

Epoch 1 Batch 0 Loss 0.8652
Epoch 1 finished, Loss: 2.8190, Time cost: 7.799777984619141 seconds

它滋养人身，也颐养人心。 >>>new>>>，，，，，，，，，，，，，，，，，，，，，，，，，。，，，，，，，，，，，，，，，，，，，，，，，，，。
Epoch 2 Batch 0 Loss 0.8603
Epoch 2 finished, Loss: 2.8023, Time cost: 9.619075536727905 seconds

这个规矩有来历。明朝的一部养生专著《遵生八笺》中说到，一位僧人，吃饭总是先淡吃三口：“第一，以知饭之正味。人食多以五味杂之，未有知正味者，若淡食，则本自甘美，初不假外味也。第二，思衣食之从来。第三，思农夫之艰苦。”>>>new>>>，，，，，，，，，，，，，，，，，，，，，，，，，。，，，，，，，，，，，，，，，，，，，，，，，，，。
Epoch 3 Batch 0 Loss 0.8521
Epoch 3 finished, Loss: 2.7380, Time cost: 9.447187185287476 seconds

一餐饭中，饭与菜，何为主，何为次?>>>new>>>，，，，，，，，，，，，，，，，，，，，，，，，，。，，，，，，，，，，，，，，，，，，，，，，，，，。
Epoch 4 Batch 0 Loss 0.8311
Epoch 4 finished, Loss: 2.6794, Time cost: 8.892837285995483 seconds

饭，的确是最养人的东西。一方水土养一方人，大江南北饭不同：南方人吃白米饭，类似于北方人吃馒头面条，也类似于西北高原吃特色面食……一餐一饭，化成人的血肉，供养着人的生命。漂泊时，它抚平你心头的创伤;安逸时，它像你初恋时分的纯洁思想。>>>new>>>，，，，，，，，，，，，，，，，，，，，，，，，，。，，，，，，，，，，，，，，，，，，，，，，，，，。
Epoch 5 Batch 0 Loss 0.8257
Epoch 5 finished, Loss: 2.6080, Time cost: 8.411806106567383 seconds

在宴席上，饭，一般被

Epoch 38 Batch 0 Loss 0.2885
Epoch 38 finished, Loss: 0.9284, Time cost: 8.411935806274414 seconds

它滋养人身，也颐养人心。 >>>new>>>，在在宴席的尾声，才姗姗上来。，称为饭，先三口，在在在饭，则自日月人心。
Epoch 39 Batch 0 Loss 0.2655
Epoch 39 finished, Loss: 0.8858, Time cost: 8.378061532974243 seconds

饭，的确是最养人的东西。一方水土养一方人，大江南北饭不同：南方人吃白米饭，类似于北方人吃馒头面条，也类似于西北高原吃特色面食……一餐一饭，化成人的血肉，供养着人的生命。漂泊时，它抚平你心头的创伤;安逸时，它像你初恋时分的纯洁思想。>>>new>>>胃肠;饭，它粮食本身的甘，也在在在宴席的也。也在在在饭，是自日月的它抚平本身的甘，也总在饭，是自日月的。
Epoch 40 Batch 0 Loss 0.2576
Epoch 40 finished, Loss: 0.7999, Time cost: 8.376927852630615 seconds

这个规矩有来历。明朝的一部养生专著《遵生八笺》中说到，一位僧人，吃饭总是先淡吃三口：“第一，以知饭之正味。人食多以五味杂之，未有知正味者，若淡食，则本自甘美，初不假外味也。第二，思衣食之从来。第三，思农夫之艰苦。”>>>new>>>自然之从来。”;在在在饭，本自甘美，是符合自然之道的味，是粮食本身的甘，其美，。
Epoch 41 Batch 0 Loss 0.2270
Epoch 41 finished, Loss: 0.7384, Time cost: 8.435436248779297 seconds

我想，三口白饭，是提醒你：食之本，在饭;饭之味，为源。饭味为正味，正味恬淡素朴。一碗白饭的味道，是百味之基。饭之甘，更在百味之上。其甘，是符合自然之道的味，是粮食本身的甘，其美，是得自日月山川的美。>>>new>>>。多多以五味杂之，它粮食本身的甘，饭也。
Epoch 42 Batch 0 Loss 0.2125
Epoch 42 finished, Loss: 0.6900, Time cost: 8.430

In [648]:
writer.save_model("language_model_short")

Model weights saved to: language_model_short


In [673]:
writer.write(content[3][:50], "。", 5)

'旧时好多有传承的人家，吃饭有个规矩：一桌子琳琅佳肴前，先吃三口白饭。长辈一代代教诲：第一口必须先吃饭，而绝不能没吃饭就夹菜。>>>new>>>。一碗白饭的味道，是百味在宴席的尾声，才姗姗上来。那时，美酒已占据胃肠;饭，往往成了陪衬。大家，也总象征性地来几口，压个轴。其甘，是符合自然之道的味，是粮食本身的甘，其美，是得自日月山川的美。'

In [681]:
writer.write("柴米油盐酱醋茶", "。", 5)

'柴米油盐酱醋茶>>>new>>>一代代教诲：第一口必须先吃饭，而绝不能没吃饭就夹菜。第三，思农夫之艰苦。”一碗是之自然之道的味，是粮食本身的甘，其美，是得自日月山川的美。恬淡。思思衣食之从来。'

尝试了长度不同的小说和文章，基本结果都是大同小异，从一开始的持续输出符号，到重复高频词语的胡言乱语，再到稍微有些意义的句子…需要的训练轮数和词表长度关系比较大，同时短一些的文章也需要训练更多的epoch，另外，输入相同时输出内容总是完全相同的，感觉不太科学，不知道需不需要加入一定随机性，而且文章过短时会发生模型背诵全文的情况…大致上感觉模型应该是可用的，只是长文章需要更多epoch的训练，时间关系就不再多做尝试了